In [27]:
import sys

from operator import add
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
import pyspark.sql.functions as f

In [45]:
spark = SparkSession\
  .builder \
  .appName("PythonWordCount") \
  .getOrCreate()


# Read CSV from OVHcloud Object Storage https://github.com/walkerkq/musiclyrics
data = spark.read.format('json').options(header='true', inferSchema='true') \
  .load('country_romance_lyrics.json', sep=",", header=True) \

print('############ JSON extract:')
data.show()


############ JSON extract:
+----------------+--------------------+--------------------+----------+
|          Artist|              Lyrics|               Title|      Year|
+----------------+--------------------+--------------------+----------+
|Billy Currington|[Verse 1]
There's...|    Let Me Down Easy|2010-01-01|
|   Blake Shelton|[Verse 1]
She lik...|Doin' What She Likes|2013-03-26|
|Billy Currington|[Verse 1]
I don't...|     I Got A Feelin'|2011-01-01|
|   Blake Shelton|[Verse 1]
Girl, I...|           Honey Bee|2011-07-11|
|     Josh Turner|[Verse 1]
Baby wh...|Why Don't We Just...|      2010|
|      David Nail|[Verse 1]
She's a...|  Whatever She's Got|2013-01-01|
|  Brett Eldredge|[Verse 1]
Well I ...|   Beat of the Music|2013-08-06|
|Billy Currington|[Verse 1-Billy Cu...|            Hey Girl|2013-01-01|
| Scotty McCreery|[Verse 1]
Girl, y...|     See You Tonight|2013-01-01|
|     Chris Young|[Verse 1]
The blu...|                 You|2011-07-11|
|  Brett Eldredge|[Verse 1]
Girl yo..

In [98]:
# Tokenize DataFrame
data = Tokenizer(inputCol="Lyrics", outputCol="Words")

In [99]:

# Transform DataFrame
reviewed = data.transform(data)
reviewed.show()

AttributeError: 'Tokenizer' object has no attribute '_jdf'

In [59]:
import pandas as pd


In [72]:
df=pd.read_json ('country_romance_lyrics.json',)

In [73]:
df.Lyrics=df.Lyrics.replace('\n', ' ')
df.head()

,Title,Artist,Year,Lyrics
0,Let Me Down Easy,Billy Currington,2010-01-01,"[Verse 1]\nThere's a little moonlight, dancing..."
1,Doin' What She Likes,Blake Shelton,2013-03-26,[Verse 1]\nShe likes it when I call in sick to...
2,I Got A Feelin',Billy Currington,2011-01-01,[Verse 1]\nI don't want to rush this thing\nI ...
3,Honey Bee,Blake Shelton,2011-07-11,"[Verse 1]\nGirl, I been thinking 'bout us\nAnd..."
4,Why Don't We Just Dance,Josh Turner,2010,[Verse 1]\nBaby why don't we just turn that TV...


In [78]:
%pip install nltk

     |████████████████████████████████| 1.4MB 2.7MB/s 
     |████████████████████████████████| 696kB 6.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434674 sha256=17a9ec152c56c204c62cd74d7dcc2e85376bb63c86d54f88fdac836dbca99a43
  Stored in directory: /Users/sherirosalia/Library/Caches/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
  Created wheel for regex: filename=regex-2020.7.14-cp37-cp37m-macosx_10_9_x86_64.whl size=285106 sha256=593d9407adaced8067ea92b88c1a6b4a6f90e23d642ae45b7a0b931e81d2d2ba
  Stored in directory: /Users/sherirosalia/Library/Caches/pip/wheels/53/55/dc/e17fa4568958f4c53be34b65e474a1327b64641f65df379ec3
Successfully built nltk regex
Note: you may need to restart the kernel to use updated packages.


In [81]:
# Import stopwords with nltk.
import nltk


In [82]:
from nltk.corpus import stopwords


In [84]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [100]:
stop = stopwords.words('english')

In [101]:
df['Lyrics'] = df['Lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.Lyrics.head()

0    [Verse 1] There's little moonlight, dancing sa...
1    [Verse 1] She likes I call sick work Spend who...
2    [Verse 1] I want rush thing I want jump gun I ...
3    [Verse 1] Girl, I thinking 'bout us And know I...
4    [Verse 1] Baby turn TV off? Three hundred fift...
Name: Lyrics, dtype: object

In [102]:
df.Lyrics.to_csv(
    'country.csv', index=True
)

In [46]:
data.withColumn('word', f.explode(f.split(f.col('Lyrics'), ' '))) \
  .groupBy('word') \
  .count() \
  .sort('count', ascending=False).show()

+----+-----+
|word|count|
+----+-----+
| the|49897|
|  to|26517|
| and|26413|
|  of|23182|
|   a|21654|
|  in|16490|
|   I|13719|
|   -|12817|
|that|10831|
|   —|10285|
| you|10140|
|with| 7642|
| for| 7423|
|  on| 7022|
|  he| 6687|
|  is| 6337|
| was| 6253|
|  it| 6179|
|  my| 5825|
| his| 5692|
+----+-----+
only showing top 20 rows



In [89]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [90]:
text_col = 'Lyrics'
review_text = data.select(text_col).filter(f.col(text_col).isNotNull())
review_text.show()

+--------------------+
|              Lyrics|
+--------------------+
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
|[Verse 1]
There's...|
+--------------------+
only showing top 20 rows



In [91]:
data=data.select("Lyrics", f.regexp_replace(f.col("Lyrics"), "[\[Verse 1]#,]", "").alias("replaced"))

In [92]:
tokenized.head()

Row(Artist='Billy Currington', Lyrics="[Verse 1]\nThere's a little moonlight, dancing on the sand\nThere's a warm breeze blowing by the ocean as you're taking my hand\nYou need to know where I'm standing now\nThat I'm right on the edge of giving in to ya\nBaby it's a long way down\n\n[Chorus]\nIf I fall, can ya let me down easy?\nIf I leave my heart with you tonight\nWill you promise me that you're going to treat it right?\nI'm barely hanging on\nIf I fall, can you let me down easy?\n\n[Verse 2]\nThe scent of your perfume, floating in the air\nLooking like an angel, lying on a blanket with a halo of hair\nAnd those lips, look to good to be true\nOnce I taste that kiss, I know what'll happen\nI'll be at the mercy of you\n\n[Chorus]\nIf I fall, can ya let me down easy?\nIf I leave my heart with you tonight\nWill you promise me that you're going to treat it right?\nI'm barely hanging on\nIf I fall, can you let me down easy?\n\nIf I fall, can ya let me down easy?\nIf I leave my heart with 

In [93]:
tokenizer = Tokenizer(inputCol="words", outputCol="words_token")


In [94]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

